# 其他工具：Grab、PyQuery


* 利用 Grab 套件的存取 HTML 資源
* 利用 PyQuery 套件的解析 HTML 格式


## 作業目標

將之前用 requests + beatifulsoup 實作的方式，改寫成 grab + pyquery，並且比較有哪些地方不同。





## requests + BeautifulSoup


In [1]:
import requests
import os

from bs4 import BeautifulSoup

In [171]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
ptt_df = pd.DataFrame()
for a in range(39214, 39200, -1):
    url = 'https://www.ptt.cc/bbs/Gossiping/index{page}.html'.format(page = a)
    resp = requests.get(url, cookies={'over18': '1'}, headers = headers)
    soup = BeautifulSoup(resp.text)

    for i in range(len(soup.find_all("div", class_="r-ent"))-1, -1, -1):
        if soup.find_all("div", class_="r-ent")[i].a == None:
            title = soup.find_all("div", class_="r-ent")[i].find("div", class_ = "title").string.strip()
        else:
            title = soup.find_all("div", class_="r-ent")[i].a.string

        # print(title)
        push = soup.find_all("div", class_="r-ent")[i].span
        push = '0' if push == None else str(push.string)


        date = soup.find_all("div", class_="date")[i].string
        author = soup.find_all("div", class_ = "author")[i].string
        if soup.find_all("div", class_ = 'title')[i].a != None:
            href = "https://www.ptt.cc" + soup.find_all("div", class_ = 'title')[i].a.get('href')
        else:
            href = "-"
        temp = (pd.DataFrame({'push': push, 'title': title, 'date': date, 'author': author, 'href': href}, index = [0]))
        ptt_df = ptt_df.append(temp, ignore_index=True)

ptt_df

,push,title,date,author,href
0,9,[問卦] 最喜歡的神奇寶貝是哪一隻？,12/06,harry7319,https://www.ptt.cc/bbs/Gossiping/M.1575597256....
1,18,[問卦] 急!黑門市場海膽這樣很貴嗎?,12/06,brooksmile,https://www.ptt.cc/bbs/Gossiping/M.1575597256....
2,2,Re: [問卦] 女生去澳洲打工為什麼要隱瞞,12/06,charlie0112,https://www.ptt.cc/bbs/Gossiping/M.1575597201....
3,61,[問卦] 卡神要改叫什麼比較好？,12/06,iversonya,https://www.ptt.cc/bbs/Gossiping/M.1575596880....
4,8,[問卦] 按喇叭的藝術,12/06,jiomi,https://www.ptt.cc/bbs/Gossiping/M.1575596844....
...,...,...,...,...,...
275,4,(本文已被刪除) [jason83813],12/06,-,-
276,19,[問卦] 買什麼東西最能看出你是財務自由?,12/06,JUNOCARE,https://www.ptt.cc/bbs/Gossiping/M.1575564313....
277,7,[問卦] 有沒有中谷美紀的八卦？,12/06,deepdish,https://www.ptt.cc/bbs/Gossiping/M.1575564273....
278,3,Re: [問卦] 說真的 FOXY時代大家都看過人獸交的片吧,12/06,rpg1510,https://www.ptt.cc/bbs/Gossiping/M.1575564156....


In [2]:
url = 'https://www.ptt.cc/bbs/Gossiping/index39210.html'
resp = requests.get(url, cookies={'over18': '1'})
soup = BeautifulSoup(resp.text)

In [30]:
for div in soup.select('div'):
    pass
print(div)

<div class="bbs-screen bbs-footer-message">本網站已依台灣網站內容分級規定處理。此區域為限制級，未滿十八歲者不得瀏覽。</div>


## Grab + PyQuery

In [46]:
from pyquery import PyQuery as pq

In [47]:
dom = pq('https://money.udn.com/money/breaknews', parser = 'html')

In [63]:
for item in dom('tr')('a').items():
    print(item.text())

港媒：大選結果有兩種，兩岸方向只有一個
行賄各國取得生意被美盯上 愛立信300億和解
2020總統電視辯論 蔡陣營拋三態度
沙烏地空軍受訓生開槍釀3死 曾批美邪惡之國
韓國瑜接民調電話說支持蔡英文 總統選票投韓國瑜
藍委陳宜民推女警蒐證曝光 質問「妳是哪個單位？滾出去！」
日本職場霸凌致死 上司涉教唆自殺遭函送
總統屏東參拜 宣布啟動六堆300年慶典籌備會
獲中國頒發名譽博士 何君堯官微：比貓熊還稀少
嫌馬桶沖不乾淨 川普指示解決水壓問題
慰勞遭藍委強推女警 徐國勇：捍衛警察尊嚴
機器人取代例行性工作後 衝著華爾街高薪工作來
欠債害妹妹800萬房產被法拍 姊坦言：是我害你那麼慘
外交部衝突藍委稱員警未表明身份 徐國勇：非事實
郭柯合破裂?柯：老闆不在 幕僚發言會「走鐘」
乾冷空氣撥雲見日 天氣風險：下周天氣大回暖
楊儒賓捐贈清大千件珍貴文物 揭1949年搶運黃金秘辛
30年薪水投入買收藏品 楊儒賓買到太太跟他劃清界線
游錫堃：台灣人要像香港 用選票傳達當家作主的意志
與宋楚瑜合作引反彈？郭台銘老虎軍團LINE群成員數腰斬
民進黨譴責國民黨：如果是男警你還敢推嗎？
「重拾公務員尊嚴」張善政：二年1,000億撥補勞保基金
統一企業鼓勵集團員工做善事 今為新住民辦耶誕趴
冷氣團持續發威冷到下周一 天氣轉乾放晴
109年總預算恐卡關 政院：盼支持福國利民政策
表現沒漏氣 花敬群：都更危老重建年底上看500案
屏東競選總部成立 蔡總統籲投票展現台灣人意志
雙北等8縣市百貨 109年元旦內用禁免洗餐具
林飛帆：譴責國民黨傷口灑鹽 施暴公務員
超萌貼圖、網紅影片 LINE、HAPPY GO讓公益變新潮
韓國瑜：外交部粗暴處理立委抗議 部長應負全責
5G手機增加電感用量 奇力新躋身前3大受惠可期
鄭文傑傳在台遭跟監 陸委會：相關單位已助維安
川普政府欲恢復聯邦死刑 最高法院擋下
超商超市化身公益平台 夜市老闆每日到門市捐數百元
單身孤單寂寞覺得冷？日本「抱抱情人」抱枕讓你有人抱
台南「中國城」重生！世界七大令人期待的公園快開放了
冷氣團持續發威 黃偉哲親自率隊探視街友
找場地開分館 館長：新竹開價是神租金
一氧化碳去年全台釀11死 消防編200萬預算補助換熱水器
環保局篩出10家疑似汙染鳳山溪工廠 稽查2違規、2續查
大批雁鴨過境台北過冬 小水鴨、花嘴鴨都現蹤
30年藝人生涯想轉行